In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# simple

In [ ]:
class SimpleResNetModel(nn.Module):
    def __init__(self):
        super(SimpleResNetModel, self).__init__()
        self.input_channels = 16

        # First Pathway
        self.cnn1 = nn.Conv1d(16, 80, kernel_size=2)
        self.dense1 = nn.Linear(80, 64)
        self.pool1 = nn.MaxPool1d(kernel_size=128)

        # Second Pathway
        self.cnn2 = nn.Conv1d(16, 64, kernel_size=2)
        self.dense2 = nn.Linear(64, 64)
        self.pool2 = nn.MaxPool1d(kernel_size=128)

        # Third Pathway
        self.cnn3 = nn.Conv1d(16, 32, kernel_size=2)
        self.dense3 = nn.Linear(32, 64)
        self.pool3 = nn.MaxPool1d(kernel_size=128)

        # Fourth Pathway
        self.cnn4 = nn.Conv1d(16, 16, kernel_size=2)
        self.dense4 = nn.Linear(16, 64)
        self.pool4 = nn.MaxPool1d(kernel_size=128)

        # Fully connected layers
        self.fc1 = nn.Linear(64 * 4, 16)
        self.fc2 = nn.Linear(16, 1)

    def forward(self, x):
        # Reshape input to match PyTorch's (batch_size, channels, sequence_length)
        x = x.permute(0, 2, 1)

        # First pathway
        x1 = F.relu(self.cnn1(x))
        x1 = x1.permute(0, 2, 1)  # Switch dimensions for Linear layer
        x1 = F.relu(self.dense1(x1))
        x1 = x1.permute(0, 2, 1)  # Switch back for pooling
        x1 = self.pool1(x1)

        # Second pathway
        x2 = F.relu(self.cnn2(x))
        x2 = x2.permute(0, 2, 1)
        x2 = F.relu(self.dense2(x2))
        x2 = x2.permute(0, 2, 1)
        x2 = self.pool2(x2)

        # Third pathway
        x3 = F.relu(self.cnn3(x))
        x3 = x3.permute(0, 2, 1)
        x3 = F.relu(self.dense3(x3))
        x3 = x3.permute(0, 2, 1)
        x3 = self.pool3(x3)

        # Fourth pathway
        x4 = F.relu(self.cnn4(x))
        x4 = x4.permute(0, 2, 1)
        x4 = F.relu(self.dense4(x4))
        x4 = x4.permute(0, 2, 1)
        x4 = self.pool4(x4)

        # Concatenate along the channel dimension
        x = torch.cat((x1, x2, x3, x4), dim=1)

        # Flatten and pass through fully connected layers
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
model = SimpleResNetModel()
print(model)

# resnet

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, filters, kernel_size=2):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(filters, filters, kernel_size=kernel_size, padding='same')
        self.bn1 = nn.BatchNorm1d(filters)
        self.conv2 = nn.Conv1d(filters, filters, kernel_size=kernel_size, padding='same')
        self.bn2 = nn.BatchNorm1d(filters)
        self.relu = nn.ReLU()

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        return self.relu(out)

class ResNetModel(nn.Module):
    def __init__(self):
        super(ResNetModel, self).__init__()
        self.input_channels = 16

        # First ResNet Pathway
        self.cnn1 = nn.Conv1d(16, 80, kernel_size=2, padding='same')
        self.res1 = ResidualBlock(80)
        self.pool1 = nn.MaxPool1d(kernel_size=128)

        # Second ResNet Pathway
        self.cnn2 = nn.Conv1d(16, 64, kernel_size=2, padding='same')
        self.res2 = ResidualBlock(64)
        self.pool2 = nn.MaxPool1d(kernel_size=128)

        # Third ResNet Pathway
        self.cnn3 = nn.Conv1d(16, 32, kernel_size=2, padding='same')
        self.res3 = ResidualBlock(32)
        self.pool3 = nn.MaxPool1d(kernel_size=128)

        # Fourth ResNet Pathway
        self.cnn4 = nn.Conv1d(16, 16, kernel_size=2, padding='same')
        self.res4 = ResidualBlock(16)
        self.pool4 = nn.MaxPool1d(kernel_size=128)

        # Fully connected layers
        self.fc1 = nn.Linear(80 + 64 + 32 + 16, 16)
        self.fc2 = nn.Linear(16, 1)

    def forward(self, x):
        # Reshape input to match PyTorch's (batch_size, channels, sequence_length)
        x = x.permute(0, 2, 1)

        # First pathway
        x1 = F.relu(self.cnn1(x))
        x1 = self.res1(x1)
        x1 = self.pool1(x1)

        # Second pathway
        x2 = F.relu(self.cnn2(x))
        x2 = self.res2(x2)
        x2 = self.pool2(x2)

        # Third pathway
        x3 = F.relu(self.cnn3(x))
        x3 = self.res3(x3)
        x3 = self.pool3(x3)

        # Fourth pathway
        x4 = F.relu(self.cnn4(x))
        x4 = self.res4(x4)
        x4 = self.pool4(x4)

        # Concatenate along the channel dimension
        x = torch.cat((x1, x2, x3, x4), dim=1)

        # Flatten and pass through fully connected layers
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
model = ResNetModel()
print(model)